In [1]:
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import matplotlib.pyplot as plt
import dicom
import os
import scipy.ndimage
import cv2
import time
import scipy.misc
import caffe
import seaborn as sns
from IPython import display
import time 

caffe_root = "./"
%matplotlib inline

size_image = (256, 216, 3)
sns.set_style("whitegrid", {'axes.grid' : False})

plt.rcParams['image.cmap'] = 'gray'

rep_dataset = "/home/atemmar/Documents/Stage_ets/caffe-release_segmentation/dataSet_preprocess/"

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def normalize_heatmap(heatmap):
    heat_map_normalize = np.zeros(heatmap.shape)
    for x in range(heatmap.shape[0]):
        for y in range(heatmap.shape[1]):
            heat_map_normalize[x,y,:] = softmax(heatmap[x,y,:])
    
    return heat_map_normalize

def do_training(solver, step_size):
        solver.step(step_size)

        heat_map = solver.test_nets[0].blobs["score-final"].data[0,:,:,:].transpose(1,2,0)
        heat_map_normalize = normalize_heatmap(heat_map)
#         heat_map_normalize = heat_map
        minimum = np.min(heat_map[:,:,0])

        plt.figure(figsize=(10,10))
        image_test = solver.test_nets[0].blobs["data"].data[0].transpose(1,2,0)
        image_test_label = solver.test_nets[0].blobs["label"].data[0,0,:,:]
        plt.subplot(1,5,1)
        plt.imshow(image_test[:,:,0])
        plt.title("image test")
        plt.subplot(1,5,2)
        plt.imshow(image_test_label)
        plt.title("Label of the test image")
        plt.subplot(1,5,3)
        plt.imshow(heat_map_normalize)
        plt.title("min : " + str(minimum))
        plt.subplot(1,5,4)
        plt.imshow(solver.test_nets[0].blobs["score"].data[0,:,:,:].transpose(1,2,0))
        plt.title("score")
        plt.subplot(1,5,5)
        plt.imshow(solver.test_nets[0].blobs["score-final"].data[0,:,:,:].transpose(1,2,0).argmax(2), vmin=0, vmax=2)
        plt.title("After : " + str(nb_step+step_size) + " itterations")
        display.display(plt.gcf())
        display.clear_output(wait=True)
        time.sleep(1)
    

/home/atemmar/anaconda2/lib/python2.7/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


# Read training set

In [2]:
# rep_traininSet = "/home/luffy/Documents/Stage/dataSet_preprocess/Train/IRM/"
# rep_label = "/home/luffy/Documents/Stage/dataSet_preprocess/Train/Segment_GT/" 

rep_traininSet = rep_dataset + "/Train/IRM/"
rep_label = rep_dataset + "/Train/Segment_GT/" 

fileList = [f for f in list(os.walk(rep_traininSet))[0][2] if ".png" in f.lower()][:1000]

nb_image = 0
images_irm = np.zeros((size_image[0], size_image[1], size_image[2], len(fileList)))
labels = np.zeros((size_image[0], size_image[1], len(fileList)))
for filename in fileList:
    images_irm[:,:, :,nb_image] = scipy.misc.imread(rep_traininSet + filename)
    labels[:,:, nb_image] = scipy.misc.imread(rep_label + filename)
    nb_image += 1
            

# Read test set

In [3]:
rep_testSet = rep_dataset + "/Test/IRM/"
rep_label_test = rep_dataset + "/Test/Segment_GT/" 

fileList = [f for f in list(os.walk(rep_testSet))[0][2] if ".png" in f.lower()]

nb_image = 0
images_irm_test = np.zeros((size_image[0], size_image[1], size_image[2], len(fileList)))
labels_test = np.zeros((size_image[0], size_image[1], len(fileList)))

for filename in fileList:
    images_irm_test[:,:, :,nb_image] = scipy.misc.imread(rep_testSet + filename)
    labels_test[:,:, nb_image] = scipy.misc.imread(rep_label_test + filename)
    nb_image += 1
            

# Training fcn perso

In [ ]:
caffe.set_device(0)
caffe.set_mode_cpu()
solver_name = caffe_root + "models/fcn_perso/solver5_topoSmoothloss.prototxt"
solver = caffe.SGDSolver(solver_name)

# for each layer, show the output shape
for layer_name, blob in solver.net.blobs.iteritems():
    print layer_name + '\t' + str(blob.data.shape)

In [ ]:
caffe.set_device(0)
caffe.set_mode_gpu()
solver_name = caffe_root + "models/fcn_perso/solver5_softmaxsmooth.prototxt"
solver = caffe.SGDSolver(solver_name)


nb_step = 0
# rep_model = "/media/luffy/DATA/Users/abdel/models_pretrained_rv_softmax/"
# rep_model = "/media/luffy/DATA/Users/abdel/models_pretrained_rv_softmax/perso1/"
rep_model  = "/home/atemmar/Documents/Stage_ets/caffe-release_segmentation/models_pretrained_rv/perso/"
# weight_softmax = rep_model + 'train_perso52_Topo_pretrained_augmented_softmax_iter_300000.caffemodel'
# solver_softmax = rep_model + 'train_perso6_Topo_pretrained_augmented_softmax_iter_100000.solverstate'
# solver_softmax = rep_model + 'train_perso5_hist_Topo_pretrained_augmented_softmaxsmooth_iter_' + str(nb_step) + '.solverstate'
# solver.restore(solver_softmax)
# solver.net.copy_from(weight_softmax)

# solver.step(1)


do_training(solver=solver, step_size=1)
# image_test = solver.test_nets[0].blobs["data"].data[0,0,:,:]
# image_test_label = solver.test_nets[0].blobs["label"].data[0,0,:,:]
# plt.subplot(1,2,1)
# plt.imshow(image_test)
# plt.title("image test")
# plt.subplot(1,2,2)
# plt.imshow(image_test_label)
# plt.title("Label of the test image")

value_loss = []

In [ ]:
solver.test_nets[0].blobs["label"].data.shape

In [ ]:
step_size = 500
for i in range(20*10):
    do_training(solver=solver, step_size=step_size)
    nb_step += step_size
#     value_loss.append(solver.test_nets[0].blobs["loss"].data)

In [ ]:
solver.test_nets[0].blobs["score-final"].data

# RESULTS

In [ ]:

caffe.set_device(0)
caffe.set_mode_gpu()
model_def = caffe_root + '/models/fcn_perso/fcn_perso5_softmaxloss.prototxt'
# model_weights = caffe_root + 'models/bvlc_alexnet_FCN_wTOPO/alexnet_LT.caffemodel'
model_weights = caffe_root + 'train_alexnet_Topo_pretrained_softmax_iter_30000.caffemodel'


# net = caffe.Net(model_def,      # defines the structure of the model
#                 model_weights, caffe.TEST)     # use test mode (e.g., don't perform dropout)
# net = solver.net


## Result on the training set

In [ ]:
import random

nb_images = 100
indices = list(range(nb_images))
images_to_test = indices[:100]

net = solver.net
for num_image_train in images_to_test:
#     img = images_irm[:,:,:,num_image_train].transpose(2,0,1)
#     label = labels[:,:, num_image_train]


#     net.blobs['data'].data[...] = img
    net.forward()
    out = net.blobs["score-final"].data[0,:,:,:].transpose(1,2,0)
    label_out = out.argmax(axis=2)
    
    plt.figure(figsize=(10,10))
    plt.subplot(1,4,1)
    plt.imshow(net.blobs['data'].data[0].transpose(1,2,0)[:,:,0])
    plt.title("Orginial image")
    plt.subplot(1,4,2)
    plt.imshow(net.blobs['label'].data[0,0,:,:], vmin=0, vmax=2)
    plt.title("Ground truth segmentation")
    plt.subplot(1,4,3)
    plt.imshow(normalize_heatmap(out))
    plt.title("Heat map")
    plt.subplot(1,4,4)
    plt.imshow(label_out, vmin=0, vmax=2)
    plt.title("Segmentation predicted")

### Result without histogram transfer

In [ ]:
model_weights = rep_model + 'train_perso52_Topo_pretrained_augmented_softmax_iter_100000.caffemodel'
net_ = caffe.Net(model_def,      # defines the structure of the model
            model_weights, caffe.TRAIN)     # use test mode (e.g., don't perform dropout)

nb_images = 100
indices = list(range(nb_images))
images_to_test = indices[:100]

net = solver.net
for num_image_train in images_to_test:
#     img = images_irm[:,:,:,num_image_train].transpose(2,0,1)
#     label = labels[:,:, num_image_train]


#     net.blobs['data'].data[...] = img
    net_.forward()
    out = net_.blobs["score-final"].data[0,:,:,:].transpose(1,2,0)
    label_out = out.argmax(axis=2)
    
    plt.figure(figsize=(10,10))
    plt.subplot(1,4,1)
    plt.imshow(net_.blobs['data'].data[0].transpose(1,2,0)[:,:,0])
    plt.title("Orginial image")
    plt.subplot(1,4,2)
    plt.imshow(net_.blobs['label'].data[0,0,:,:], vmin=0, vmax=2)
    plt.title("Ground truth segmentation")
    plt.subplot(1,4,3)
    plt.imshow(normalize_heatmap(out))
    plt.title("Heat map")
    plt.subplot(1,4,4)
    plt.imshow(label_out, vmin=0, vmax=2)
    plt.title("Segmentation predicted")

## Results on the test set

In [ ]:
##### import random

nb_images = 25
indices = list(range(nb_images))
images_to_test = indices

net = solver.test_nets[0]
for num_image_test in images_to_test:
#     img = images_irm_test[:,:,:,num_image_test].transpose(2,0,1)
#     label = labels_test[:,:, num_image_test]

#     net.blobs['data'].data[...] = img
    net.forward()
    
    out = net.blobs["score-final"].data[0,:,:,:].transpose(1,2,0)
    label_out = out.argmax(axis=2)
    
    plt.figure(figsize=(10,10))
    plt.subplot(1,4,1)
    plt.imshow(net.blobs['data'].data[0].transpose(1,2,0)[:,:,0])
    plt.title("Orginial image")
    plt.subplot(1,4,2)
    plt.imshow(net.blobs['label'].data[0,0,:,:], vmin=0, vmax=2)
    plt.title("Ground truth segmentation")
    plt.subplot(1,4,3)
    plt.imshow(normalize_heatmap(out))
    plt.title("Heat map")
    plt.subplot(1,4,4)
    plt.imshow(label_out, vmin=0, vmax=2)
    plt.title("Segmentation predicted")

In [ ]:
# the parameters are a list of [weights, biases]
net = solver.net
conv1 = net.blobs['score'].data

# plt.figure(figsize=(30,30))
plt.figure(figsize=(20,90))
nb_c = min(conv1.shape[1],10)
for i in range(conv1.shape[1]):
    plt.subplot(20,nb_c,i+1)
#     plt.figure(figsize=(10,10))
    plt.imshow(conv1[0,i,:,:], cmap='jet')
    plt.axis('off')



In [ ]:
plt.figure()
plt.imshow(net.blobs['score'].data[0,1:].argmax(0))

In [ ]:
net.blobs['conv9'].data[0].transpose(1,2,0)

### Depending on the number of itterations done

#### Load all the nets

In [ ]:
nb_itterations = [2000, 20000, 60000, 100000, 140000, 180000, 200000, 220000, 260000, 300000]

model_def = caffe_root + 'models/fcn_perso/fcn5_deploy_softmaxloss.prototxt'
rep_model = "/home/atemmar/Documents/Stage_ets/caffe-release_segmentation/models_pretrained_rv/perso/"


weight_softmax = rep_model + 'train_perso52_Topo_pretrained_augmented_softmax_iter_300000.caffemodel'

nets = []

# solver_name = caffe_root + "models/fcn_perso/solver5_softmax.prototxt"
# solver = caffe.SGDSolver(solver_name)

caffe.set_device(0)
caffe.set_mode_gpu()
for nb_itter in nb_itterations:
    model_weights = rep_model + 'train_perso52_Topo_pretrained_augmented_softmax_iter_' + str(nb_itter) +'.caffemodel'
#     solver.restore(model_solver)
#     net = solver.net
    net = caffe.Net(model_def,      # defines the structure of the model
                model_weights, caffe.TEST)     # use test mode (e.g., don't perform dropout)
    nets.append(net)

In [ ]:
nbs_nets = len(nets)
nb_columns_display = 2 + nbs_nets

for num_image_test in range(images_irm_test.shape[3]):
    img = images_irm_test[:,:,0,num_image_test]
    
    max_img, min_img = np.max(img), np.min(img)
    img = 2*(img-max_img)/(max_img - min_img) - 1
    
    label = labels_test[:,:, num_image_test]

    plt.figure(figsize=(30,30))
    plt.subplot(1,nb_columns_display,1)
    plt.imshow(img)
    plt.title("Orginial image")
    plt.subplot(1,nb_columns_display,2)
    plt.imshow(label, vmin=0, vmax=2)
    plt.title("Ground truth segmentation")
    for i, net in enumerate(nets):
        net.blobs['data'].data[...] = img
        net.forward()
        out = net.blobs["score-final"].data[0,:,:,:].transpose(1,2,0)
        label_out = out.argmax(axis=2)
        plt.subplot(1,nb_columns_display,2+i+1)
        plt.imshow(normalize_heatmap(out))
        plt.title("Segmentation predicted after " + str (nb_itterations[i]))

In [ ]:
nets[-1].blobs['data'].data[...] = img
nets[-1].forward()
out = nets[-1].blobs["score-final"].data[0,:,:,:].transpose(1,2,0)

In [ ]:
np.sum(out.argmax(axis=2))

#### Pixel wise accuaracy depending on the number of itterations done

In [ ]:
accuaracy_depending_nbItterations = np.zeros(len(nets), dtype=np.float)

def compute recall(label, label_predicted):
    right_prediction = (label == label_predicted).astype(int)
    nb_pos_predict = nb.sum(right_prediction[label != 0])
    nb_pos = np.sum((label != 0).astype(int))
    
    return float(nb_pos_predict/nb_pos)

for num_image_test in range(images_irm_test.shape[3]):
    img = images_irm_test[:,:,:,num_image_test].transpose(2,0,1)
    label = labels_test[:,:, num_image_test]
    for i, net in enumerate(nets):
        net.blobs['data'].data[...] = img
        net.forward()
        out = net.blobs["score-final"].data[0,:,:,:].transpose(1,2,0)
        label_predicted = out.argmax(axis=2)
        accuaracy_depending_nbItterations[i] += np.sum((label == label_predicted).astype(int))/float(label.size)

accuaracy_depending_nbItterations /= images_irm_test.shape[3]
        

In [ ]:
plt.plot(nb_itterations, accuaracy_depending_nbItterations)
plt.title("Pixel wise accuaracy depending on the number of itterations done")

In [ ]:
# for each layer, show the output shape
for layer_name, blob in net.blobs.iteritems():
    print layer_name + '\t' + str(blob.data.shape)